In [ ]:
import pandas as pd
import numpy as np
import requests
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
"""
    READ
"""

with sqlite3.connect('qiita_data.db') as con:
    df_attribute = pd.read_sql_query("SELECT id, title, url FROM PAGE_ATTRIBUTE", con=con).set_index("id")
    df_embedding = pd.read_sql_query("SELECT * FROM TITLE_EMBEDDING", con=con).set_index("id")

In [ ]:
"""
    対象単語の埋め込みを抽出
"""

target_word = "Docker"

target_embedding = requests.post('http://fasttext-vector-api:80/', json=[target_word])
target_embedding = np.array(list(target_embedding.json().values()))
target_embedding = target_embedding.mean(axis=0).reshape(1, -1)
target_embedding

In [ ]:
"""
    タイトルと単語のコサイン類似度
"""

cos_sim = cosine_similarity(target_embedding, df_embedding.to_numpy())[0]

"""
    類似度Top10の属性情報を抽出
"""

TopK_indices = np.argsort(-cos_sim)[:10]
TopK_ids = df_attribute.iloc[TopK_indices].index
TopK_attributes = df_attribute.loc[TopK_ids]
list(TopK_attributes.T.to_dict().values())